# GAT

This notebook accompanies my blog on Graph Neural Networks and takes inspiration from PyTorch Geometric tutorial.

In [1]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-geometric
!pip install torch-cluster -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-1.10.0+cu111.html

Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 7.9 MB 28.3 MB/s 
Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 3.5 MB 32.5 MB/s 
     |████████████████████████████████| 370 kB 21.3 MB/s 
     |████████████████████████████████| 482 kB 61.0 MB/s 
     |████████████████████████████████| 41 kB 766 kB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.3-py3-none-any.whl size=581968 sha256=aa0b1b280fa7567994f5541c7a74f28629525b966882568f2f02e75d70aabad7
  Stored in directory: /root/.cache/pip/wheels/c3/2a/58/87ce0508964d4def1aafb92750c4f3ac77038efd1b9a89dcf5
Successfully built torch-geometric
Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 2.3 MB 26.3 MB/s 
Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 747 kB 41.7 MB/s 


In [ ]:
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GATConv

In [ ]:
dataset = 'Cora'
path = "."
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]

Processing...
Done!


In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv1 = GATConv(in_channels, 8, heads=8, dropout=0.6)
        self.conv2 = GATConv(8 * 8, out_channels, heads=1, concat=False, dropout=0.6)
        
    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=-1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dataset.num_features, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

In [ ]:
def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


@torch.no_grad()
def test(data):
    model.eval()
    out, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        acc = float((out[mask].argmax(-1) == data.y[mask]).sum() / mask.sum())
        accs.append(acc)
    return accs


for epoch in range(1, 201):
    train(data)
    train_acc, val_acc, test_acc = test(data)
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}')

Epoch: 001, Train: 0.2214, Val: 0.1360, Test: 0.1330
Epoch: 002, Train: 0.1786, Val: 0.1140, Test: 0.1060
Epoch: 003, Train: 0.2429, Val: 0.1300, Test: 0.1340
Epoch: 004, Train: 0.3000, Val: 0.1580, Test: 0.1650
Epoch: 005, Train: 0.3214, Val: 0.1640, Test: 0.1790
Epoch: 006, Train: 0.3357, Val: 0.1700, Test: 0.1960
Epoch: 007, Train: 0.4357, Val: 0.2340, Test: 0.2380
Epoch: 008, Train: 0.5429, Val: 0.2920, Test: 0.3140
Epoch: 009, Train: 0.6500, Val: 0.3860, Test: 0.3890
Epoch: 010, Train: 0.7500, Val: 0.4600, Test: 0.4880
Epoch: 011, Train: 0.8286, Val: 0.5760, Test: 0.5820
Epoch: 012, Train: 0.8643, Val: 0.6620, Test: 0.6440
Epoch: 013, Train: 0.8714, Val: 0.6940, Test: 0.6670
Epoch: 014, Train: 0.8786, Val: 0.7140, Test: 0.6890
Epoch: 015, Train: 0.8714, Val: 0.7040, Test: 0.6900
Epoch: 016, Train: 0.8857, Val: 0.7240, Test: 0.7140
Epoch: 017, Train: 0.9000, Val: 0.7300, Test: 0.7310
Epoch: 018, Train: 0.9000, Val: 0.7440, Test: 0.7530
Epoch: 019, Train: 0.9071, Val: 0.7740, Test: 